<a href="https://colab.research.google.com/github/kairamilanifitria/NLP-Projects/blob/main/Project%202%20Text%20Summarization/2_Dev_T5_Abstractive%2BExtractive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

# Load the dataset
dev_df = pd.read_csv('/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/fine-tune_dataset/dev_df.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/fine-tune_dataset/test_df.csv')

# Basic overview
print(dev_df.shape)
print(dev_df.info())
print(dev_df.isnull().sum())

print(test_df.shape)
print(test_df.info())
print(test_df.isnull().sum())


(9540, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9540 entries, 0 to 9539
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   9540 non-null   int64 
 1   url                  9540 non-null   object
 2   text                 9540 non-null   object
 3   abstractive_summary  9540 non-null   object
 4   extractive_summary   9540 non-null   object
dtypes: int64(1), object(4)
memory usage: 372.8+ KB
None
id                     0
url                    0
text                   0
abstractive_summary    0
extractive_summary     0
dtype: int64
(9775, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9775 entries, 0 to 9774
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   9775 non-null   int64 
 1   url                  9775 non-null   object
 2   text                 9775 non

In [5]:
dev_df = dev_df.drop(['id', 'url'], axis=1)
test_df = test_df.drop(['id', 'url'], axis=1)

In [6]:
dev_df.iloc[1]

,1
text,liputan6 com surabaya tokoh madura dan kaliman...
abstractive_summary,gubernur jawa timur minta para gubernur se kal...
extractive_summary,hingga kini ada sekitar 90 ribu ungsi masuk ke...


In [7]:
!pip install datasets transformers torch accelerate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.9 MB/s eta 0:00:00


In [8]:
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EncoderDecoderModel
from transformers import pipeline

In [9]:
# Check if GPU is available and set device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [11]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_path = "cahya/t5-base-indonesian-summarization-cased"

tokenizer = T5Tokenizer.from_pretrained(model_path)

model = T5ForConditionalGeneration.from_pretrained("cahya/t5-base-indonesian-summarization-cased").to(device)

tokenizer_config.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/793k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [16]:
# make a code to training model text summarization using the data in dev_df

from transformers import TrainingArguments, Trainer

# Convert pandas DataFrame to Hugging Face Dataset
dev_dataset = Dataset.from_pandas(dev_df)

# Convert pandas DataFrame to Hugging Face Dataset
dev_dataset = Dataset.from_pandas(dev_df)

def preprocess_function(examples):
    # Use a separator token to distinguish between the two types of summaries
    separator_token = " <|extractive|> "

    # Convert list summaries to strings if necessary
    extractive_summary = [
        " ".join(summary) if isinstance(summary, list) else summary
        for summary in examples["extractive_summary"]
    ]  # List comprehension for handling list summaries
    abstractive_summary = [
        " ".join(summary) if isinstance(summary, list) else summary
        for summary in examples["abstractive_summary"]
    ]  # List comprehension for handling list summaries

    # Combine both summaries into one string with a separator for each example in the batch
    combined_summary = [
        ex_sum + separator_token + ab_sum
        for ex_sum, ab_sum in zip(extractive_summary, abstractive_summary)
    ]

    # Tokenize the text input
    model_inputs = tokenizer(
        examples["text"], max_length=512, truncation=True, padding="max_length"
    )

    # Tokenize the combined summary (labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            combined_summary, max_length=256, truncation=True, padding="max_length"
        )

    # Add the tokenized labels to the model inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Apply preprocessing to the dataset
tokenized_dev_datasets = dev_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/9540 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [17]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
    push_to_hub=False,
    fp16=True,
)

# Define data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dev_datasets,
    eval_dataset=tokenized_dev_datasets,
    data_collator=data_collator,
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
1,0.414500,0.354444
2,0.378800,0.332050
3,0.363100,0.325627


TrainOutput(global_step=3579, training_loss=0.42165286076671754, metrics={'train_runtime': 3829.0858, 'train_samples_per_second': 7.474, 'train_steps_per_second': 0.935, 'total_flos': 1.74283748278272e+16, 'train_loss': 0.42165286076671754, 'epoch': 3.0})

In [18]:
trainer.save_model("./trained_model")

In [19]:
import os

# Define the path in Google Drive where you want to save the model
output_dir = '/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/abstractive+extractive_T5'

# Create the directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the trained model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model saved to {output_dir}")

Model saved to /content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/abstractive+extractive_T5
